# Storage

- Storage blob : 오디오, 비디오, 이미지 등과 같은 비정형 데이터를 저장해야 하는 경우 (이진 데이터 특화)
    - container : 폴더 정도로 생각하면 됨 
    - blob : 폴더 속 파일 정도로 생각하면 됨 
    - BlobServiceClient : 모든 접근을 가질 수 있기 때문에 이것으로 사용
- Storage que : 메시지 저장소
- Storage table : 구조화된 데이터의 스키마 없는 스토리지를 위한 NoSQL 스토리지 
- Storage disk : Azure VM용 블록 수준 스토리지 볼륨
- Storage file : 파일 자체를 공유하는 듯?? (NAS처럼)

- storage db 연결

In [46]:
from azure.storage.blob import BlobServiceClient, BlobClient

### storage connect str (고유키 값)
connect_str = 'DefaultEndpointsProtocol=https;AccountName=hnu;AccountKey=hGGsXvXa+lwezkaGaKj8EYYSLsYA7sp3oPI1u7Oct0si3oi9QLawtpG1Fihsc603PT5+i0UZZEGhyc/SgUc2cA==;EndpointSuffix=core.windows.net'
key = 'hGGsXvXa+lwezkaGaKj8EYYSLsYA7sp3oPI1u7Oct0si3oi9QLawtpG1Fihsc603PT5+i0UZZEGhyc/SgUc2cA=='

### storage connect
client = BlobServiceClient.from_connection_string(connect_str)

### 01. Container

- container 출력

In [47]:
[i.name for i in client.list_containers()]

['data', 'ex1', 'rawdata']

- container 생성 
    - *container가 있으면 오류를 냄으로 try except 문을 이용하는게 좋을 듯*
    - *_같은 특수 문자 안됨..*

In [39]:
try:
    client.create_container("test")
except:
    pass

- container 삭제
    - *container 명을 명시 해줘야 좋을듯 안그러면 접근 중인 container 삭제하기 때문에 헷갈릴 수 있을 듯*

In [37]:
client.delete_container("test")

- 원하는 container 접근

In [48]:
data_container = client.get_container_client("data")

- container blob 나열 
    - *해당 함수는 데이터 자체를 품고 있기는 않음, blob(file) 이름 정도만 얻어가면 될듯*
    - *container list 얻는것 처럼 iterator이기 때문에 순회하면 사라짐*

In [49]:
[i.name for i in data_container.list_blobs()]

['20220216.csv', 'ex.json', 'trips.json']

### 02. Blob

- 원하는 blob 연결

In [7]:
trips_blob = data_container.get_blob_client('20220216.csv')

- csv 파일 읽기
    - *반환된 데이터 모두 문자형 주의*

In [8]:
def read_blob_csv(blob):
    import pandas as pd
    
    blob_data = blob.download_blob()
    try:    # 기본적으로 utf-8로 인코딩 접근 
        data = blob_data.content_as_text()
        data = data.split("\n")
    except: # encoding cp949로 접근
        data = blob_data.content_as_text(encoding="cp949")
        data = data.split("\n")
    data = list(map(lambda data: data.split(",") ,data))
    col = data[0]
    content = data[1:]
    data = pd.DataFrame(content, columns=col)
    #split 때문에 생겨난 마지막 nan 값 제거
    data = data.dropna(axis=0)
    return data

In [9]:
data = read_blob_csv(trips_blob)
data.head()

,no,cartype,startpos1,startpos2,endpos1,endpos2,receipttime_date,receipttime_time,settime_date,settime_time,ridetime_date,ridetime_time
0,2560,중형승용,송파구,잠실7동,중구,필동,2022-02-16,06:57:00,2022-02-16,07:12:03,2022-02-16,07:26:30
1,1375,중형승합,강서구,화곡제4동,성북구,월곡제2동,2022-02-16,07:00:00,2022-02-16,07:09:31,2022-02-16,07:26:28
2,6400,중형승용,마포구,연남동,영등포구,여의동,2022-02-16,07:00:00,2022-02-16,06:55:21,2022-02-16,07:19:15
3,8019,중형 승합,노원구,상계8동,강북구,인수동,2022-02-16,07:00:00,2022-02-16,07:05:38,2022-02-16,07:23:20
4,2560,중형승용,송파구,거여2동,강남구,역삼2동,2022-02-16,07:00:00,2022-02-16,06:37:06,2022-02-16,06:47:12


- json 파일 읽기 

In [50]:
trips_blob = data_container.get_blob_client('trips.json')

In [61]:
def read_blob_json(blob):
    import json 
    blob_data = blob.download_blob()
    data = json.loads(blob_data.content_as_bytes())
    return data

In [ ]:
data = read_blob_json(trips_blob)
data

- blob 생성
    - *local에 저장하지 않고 바로 저장 알아보기..*
    - *json*
    - *csv*

In [41]:
import json 

blob = BlobClient.from_connection_string(conn_str=connect_str, container_name="data", blob_name="test.json")
blob.upload_blob(json.dumps(data))

{'etag': '"0x8DA322AE33E3B9D"',
 'last_modified': datetime.datetime(2022, 5, 10, 2, 15, tzinfo=datetime.timezone.utc),
 'content_md5': bytearray(b'\xce\x11NE\x01\xd2\xf4\xe2\xdc\xea>\x17\xb5F\xf39'),
 'client_request_id': 'fe413eee-d006-11ec-84a6-00155d76df93',
 'request_id': '77b56737-601e-0086-0813-643576000000',
 'version': '2021-04-10',
 'version_id': None,
 'date': datetime.datetime(2022, 5, 10, 2, 15, tzinfo=datetime.timezone.utc),
 'request_server_encrypted': True,
 'encryption_key_sha256': None,
 'encryption_scope': None}

- blob 삭제
    - *원하는 blob 접근 뒤 delete_blob() 실행*

In [13]:
test_blob = data_container.get_blob_client('test.json')

In [14]:
test_blob.delete_blob()